In [18]:
import numpy as np
import pandas as pd
import glob
import yaml
import yfinance as yf
import matplotlib.pyplot as plt
import json
import os

anonymized = int(os.getenv('PFANALYZER_ANONYMIZED', '0')) != 0

dfs = []
for fname in glob.glob('/app/data/*.csv'):
    dfs.append(pd.read_csv(fname))
df = pd.concat(dfs)

df = df.groupby(['account', 'asset']).sum()
df = df.sort_index()

df.to_pickle('/tmp/df_data.pkl')
display(df)

with open('/app/pfanalyzer.yml') as f:
    config = yaml.safe_load(f.read())
print(config)

amount
account     asset                    
nisa        emaxis_slim_acwi     1000
nisa_growth emaxis_slim_acwi     1000
sbi         1301.T               1000
            2036.T               1000
            2255.T               1000
            NVDA                 1000
            SPY                  1000
            TQQQ                 1000
            br_mmf               1000
            emaxis_slim_acwi     1000
            emaxis_slim_sp500    1000
            gs_mmf               1000
            jpy                  1000
            nomura_nobl          1000
            usd                  1000
wallet      btc                     1

{'assets': {'emaxis_slim_sp500': {'equivalent': 'SPY', 'equivalent_scale': 27825.4209446}, 'emaxis_slim_acwi': {'equivalent': 'ACWI', 'equivalent_scale': 6914.77577116}, 'nomura_nobl': {'equivalent': 'NOBL', 'equivalent_scale': 5604.49454688}, 'br_mmf': {'equivalent': 'usd', 'equivalent_scale': 1}, 'gs_mmf': {'equivalent': 'usd', 'equivalent_scale': 100}, '2036.T': {'leverage': 2, 'base': 'IAU'}, '2255.T': {'equivalent': 'TLT'}, '2256.T': {'equivalent': 'AGG'}, '2257.T': {'equivalent': 'LQD'}, '2258.T': {'equivalent': 'HYG'}, 'IAU': {'equivalent': 'GLD'}, 'TLT': {'asset_class': 'bond'}, 'AGG': {'asset_class': 'bond'}, 'LQD': {'asset_class': 'bond'}, 'HYG': {'asset_class': 'bond'}, 'GLD': {'asset_class': 'commodity'}, 'btc': {'asset_class': 'crypto'}, 'eth': {'asset_class': 'crypto'}, 'usd': {'asset_class': 'cash'}, 'jpy': {'asset_class': 'cash'}, 'ACWI': {'market_portfolio': True}, 'SPY': {'market_portfolio': True}}}


In [19]:
def get_prices(df):
    start_date = "2024-01-01"
    end_date = None

    tickers = list(df.index.get_level_values('asset'))
    for k, v in config['assets'].items():
        if 'equivalent' in v:
            tickers += [k, v['equivalent']]
    tickers += ['JPY=X', 'BTC-USD', 'ETH-USD']
    tickers = list(set(tickers))
    tickers = [x for x in tickers if x not in ['jpy', 'usd', 'btc', 'eth']]

    df_yf = yf.download(tickers, start=start_date, end=end_date)
    cl = df_yf['Close']
    
    d = {}
    d['usd'] = 1.0
    d['jpy'] = 1.0 / cl['JPY=X'].dropna().iloc[-1]
    d['btc'] = cl['BTC-USD'].dropna().iloc[-1]
    d['eth'] = cl['ETH-USD'].dropna().iloc[-1]
    
    for t in cl.columns:
        x = cl[t].dropna()
        if x.shape[0] == 0:
            continue
        d[t] = x.iloc[-1]
        if t[-2:] == '.T':
            d[t] *= d['jpy']

    return d
    
df = pd.read_pickle('/tmp/df_data.pkl')

prices = get_prices(df)
with open('/tmp/prices.json', 'w') as f:
    json.dump(prices, f)
print(prices)

[*********************100%%**********************]  25 of 25 completed

5 Failed downloads:
['GS_MMF', 'EMAXIS_SLIM_SP500', 'NOMURA_NOBL', 'BR_MMF', 'EMAXIS_SLIM_ACWI']: Exception('%ticker%: No timezone found, symbol may be delisted')


{'usd': 1.0, 'jpy': 0.006647168198428357, 'btc': 51577.171875, 'eth': 3016.054931640625, '1301.T': 23.597447104420667, '2036.T': 264.2914075695115, '2255.T': 1.3865993267632504, '2256.T': 1.3693166488762416, '2257.T': 1.3786226437829463, '2258.T': 1.383275742664036, 'ACWI': 106.7699966430664, 'AGG': 97.33999633789062, 'BTC-USD': 51577.171875, 'ETH-USD': 3016.054931640625, 'GLD': 188.6199951171875, 'HYG': 77.3499984741211, 'IAU': 38.52000045776367, 'JPY=X': 150.44000244140625, 'LQD': 108.05999755859375, 'NOBL': 96.86000061035156, 'NVDA': 788.1699829101562, 'SPY': 507.8500061035156, 'TLT': 93.87000274658203, 'TQQQ': 59.619998931884766}


In [21]:
df = pd.read_pickle('/tmp/df_data.pkl')

df = df.reset_index()
for k, v in config['assets'].items():
    if 'equivalent' in v:
        idx = df['asset'] == k
        df.loc[idx, 'asset'] = v['equivalent']
        if 'equivalent_scale' in v:
            df.loc[idx, 'amount'] /= v['equivalent_scale']
        else:
            df.loc[idx, 'amount'] *= prices[k] / prices[v['equivalent']]

df = df.groupby(['account', 'asset']).sum()
df = df.sort_index()

df['price'] = df.index.get_level_values('asset').to_series().map(prices).values

if 'amount_usd' not in df.columns:
    df['amount_usd'] = 0.0
df['amount_usd'] = df['amount_usd'].fillna(0)

idx = ~df['amount'].isna()
df.loc[idx, 'amount_usd'] += df.loc[idx, 'amount'] * df.loc[idx, 'price']

if 'amount_jpy' in df.columns:
    idx = ~df['amount_jpy'].isna()
    df.loc[idx, 'amount_usd'] += df.loc[idx, 'amount_jpy'] * prices['jpy']

df = df[['price', 'amount_usd']]

if anonymized:
    df = df.reset_index()
    df['account'] = 'anonymous'
    df = pd.concat([
        df.groupby(['account', 'asset'])['price'].mean(),
        df.groupby(['account', 'asset'])['amount_usd'].sum(),
    ], axis=1)
    df['amount_usd'] /= df['amount_usd'].sum()

df.to_pickle('/tmp/df_processed.pkl')
display(df)

price     amount_usd
account     asset                              
nisa        ACWI      106.769997      15.440847
nisa_growth ACWI      106.769997      15.440847
sbi         1301.T     23.597447   23597.447104
            2036.T    264.291408  264291.407570
            ACWI      106.769997      15.440847
            NOBL       96.860001      17.282558
            NVDA      788.169983  788169.982910
            SPY       507.850006  507868.257400
            TLT        93.870003    1386.599327
            TQQQ       59.619999   59619.998932
            jpy         0.006647       6.647168
            usd         1.000000    2010.000000
wallet      btc     51577.171875   51577.171875